In [58]:
import pandas as pd
import matplotlib.pyplot as plt

import urllib
import urllib.request
from urllib.parse import unquote

import json

import requests
import pprint

In [11]:
_5min = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/한국전력거래소_5분단위 전력수급현황_20220407.csv', encoding='euc-kr')
solar = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/한국전력거래소_지역별 시간별 태양광 발전량_20230228.csv', encoding='euc-kr')

In [52]:
# plt.plot(_5min['현재수요(MW)'])
_5min

,기준일시,공급능력(MW),현재수요(MW),최대예측수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00,64355.0,51726.33,59400,12628.67,24.41439,12628.67,24.41439
1,2012-06-01 00:05,64355.0,51342.53,60800,13012.47,25.34442,13012.47,25.34442
2,2012-06-01 00:10,64355.0,51583.17,60800,12771.83,24.75969,12771.83,24.75969
3,2012-06-01 00:15,64355.0,51631.69,60800,12723.31,24.64245,12723.31,24.64245
4,2012-06-01 00:20,64355.0,51252.66,60800,13102.35,25.56423,13102.35,25.56423
...,...,...,...,...,...,...,...,...
1035062,2022-04-07 23:35,82791.9,63877.30,69300,18914.70,29.61100,10803.00,16.91210
1035063,2022-04-07 23:40,82768.1,63525.30,69300,19242.80,30.29160,10696.40,16.83800
1035064,2022-04-07 23:45,82767.9,63506.20,69300,19261.70,30.33040,10501.30,16.53580
1035065,2022-04-07 23:50,82762.4,62819.60,69300,19942.80,31.74610,10706.30,17.04290


In [30]:
# (_5min.isnull().sum()/ _5min.shape[0]).sort_values(ascending=False)
(solar.isnull().sum()/ solar.shape[0]).sort_values(ascending=False)

 풍력 발전량(MWh)      0.352941
거래일자              0.000000
거래시간              0.000000
지역                0.000000
 태양광 발전량(MWh)     0.000000
dtype: float64

In [28]:
_5min.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035067 entries, 0 to 1035066
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   기준일시        1035067 non-null  object 
 1   공급능력(MW)    1035067 non-null  float64
 2   현재수요(MW)    1035067 non-null  float64
 3   최대예측수요(MW)  1035067 non-null  int64  
 4   공급예비력(MW)   1035067 non-null  float64
 5   공급예비율(퍼센트)  1035067 non-null  float64
 6   운영예비력(MW)   1035067 non-null  float64
 7   운영예비율(퍼센트)  1035067 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 63.2+ MB


In [31]:
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918000 entries, 0 to 917999
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   거래일자            918000 non-null  object
 1   거래시간            918000 non-null  int64 
 2   지역              918000 non-null  object
 3    태양광 발전량(MWh)   918000 non-null  object
 4    풍력 발전량(MWh)    594000 non-null  object
dtypes: int64(1), object(4)
memory usage: 35.0+ MB


In [40]:
solar[' 태양광 발전량(MWh) ']
# solar.columns

0             
1             
2             
3             
4             
          ... 
917995    3.59
917996    3.46
917997    2.71
917998    0.82
917999    0.18
Name:  태양광 발전량(MWh) , Length: 918000, dtype: object

In [43]:
solar

,거래일자,거래시간,지역,태양광 발전량(MWh),풍력 발전량(MWh)
0,2017-01-01,1,경기도,,0.01
1,2017-01-01,2,경기도,,
2,2017-01-01,3,경기도,,
3,2017-01-01,4,경기도,,0.01
4,2017-01-01,5,경기도,,0.02
...,...,...,...,...,...
917995,2023-02-28,20,부산시,3.59,
917996,2023-02-28,21,부산시,3.46,
917997,2023-02-28,22,부산시,2.71,
917998,2023-02-28,23,부산시,0.82,


https://apihub.kma.go.kr/
<br>예특보

https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?authKey=yEtC0d2xQmKLQtHdsWJi5g

In [55]:
'''
domain = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?'
tmfc1tmfc2 = ''
tmfe1tmfe2 = ''
disp = 'disp=1&'
auth = 'authKey=yEtC0d2xQmKLQtHdsWJi5g'

url = domain + tmfc1tmfc2 + tmfe1tmfe2 + disp + auth

with urlopen(url) as f:
    html = f.read()
    print(html)

html.decode('e-kr')
'''

b'#START7777\n# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF\n11A00101,202306080500,202306080000,A02,0,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4,SE,1,S,-99,60,DB04,1,\xc8\xe5\xb8\xae\xb0\xed \xb0\xa1\xb2\xfb \xba\xf1,=\n11A00101,202306080500,202306081200,A02,1,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4,SW,1,W,20,60,DB03,1,\xb1\xb8\xb8\xa7\xb8\xb9\xb0\xed \xc7\xd1\xb6\xa7 \xba\xf1,=\n11A00101,202306080500,202306090000,A02,2,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4,SE,1,S,15,0,DB01,0,\xb8\xbc\xc0\xbd,=\n11A00101,202306080500,202306091200,A02,3,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4,S,1,SW,21,20,DB03,0,\xb1\xb8\xb8\xa7\xb8\xb9\xc0\xbd,=\n11A00101,202306080500,202306100000,A02,4,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4,S,1,SW,15,30,DB04,0,\xc8\xe5\xb8\xb2,=\n11A00101,202306080500,202306101200,A02,5,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4,SE,1,S,20,30,DB04,0,\xc8\xe5\xb8\xb2,=\n11A00101,202306080500,202306110000,A02,6,109,2,shs715,\xbd\xc5\xc7\xf6\xbd\xc4

단기예보특보

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=dN%2FllUgYKJLRNhR0021nUWL9K1mw%2BVGerpz3ROJEVnDbOKfmQt20uESkPz2w9ccaRfHG8dHu6g7%2Bg14HnOZ1tA%3D%3D&pageNo=1&numOfRows=1000

In [126]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
params ={'serviceKey' : 'dN%2FllUgYKJLRNhR0021nUWL9K1mw%2BVGerpz3ROJEVnDbOKfmQt20uESkPz2w9ccaRfHG8dHu6g7%2Bg14HnOZ1tA%3D%3D', 
         'pageNo' : '1', 
         'numOfRows' : '1000', 
         'dataType' : 'xml'}
#          , 
#          'base_date' : '20210628', 
#          'base_time' : '0600', 
#          'nx' : '55', 
#          'ny' : '127'}

response = requests.get(url, params=params)
print(response.content)


b'<OpenAPI_ServiceResponse>\n\t<cmmMsgHeader>\n\t\t<errMsg>SERVICE ERROR</errMsg>\n\t\t<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>\n\t\t<returnReasonCode>30</returnReasonCode>\n\t</cmmMsgHeader>\n</OpenAPI_ServiceResponse>'


In [127]:
response

<Response [200]>

In [120]:

# url 불러오기
response = requests.get(url)

#데이터 값 출력해보기
content = response.text


In [121]:
content

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>01</resultCode><resultMsg>APPLICATION_ERROR</resultMsg></header></response>\r\n'

In [110]:
type(content)

str

In [111]:
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

In [122]:
#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('header') # item 으로 구분되어 있는 경우도 있음
print(rows)

[<header><resultCode>01</resultCode><resultMsg>APPLICATION_ERROR</resultMsg></header>]
